# Package loading

In [1]:
import tensorflow as tf
# from tensorflow.keras.models import *
# from tensorflow.keras.layers import Input, Conv2D, UpSampling2D, BatchNormalization, Activation, add, concatenate

In [2]:
tf.version

<module 'tensorflow._api.v2.version' from 'C:\\OSGEO4~1\\apps\\Python37\\lib\\site-packages\\tensorflow\\_api\\v2\\version\\__init__.py'>

# Datasets Manipulations

In [3]:
import os
from glob import glob
import numpy as np
from osgeo import gdal
from matplotlib import pyplot as plt

In [4]:
# data path
# image_path = u'../../tiles/'
# mask_path = u'../../masks/'
# images = [image_path + i for i in os.listdir(image_path)]
# masks = [mask_path + i for i in os.listdir(mask_path)]
path = u'../../'

In [5]:
def load_data(path):
    images_path = glob(os.path.join(path, "tiles/*"))
    masks_path = glob(os.path.join(path, "masks/*"))
    return images_path, masks_path

In [ ]:
# def load_data(path, patch_size, num):
#     images_path = glob(os.path.join(path, "tiles/*"))
#     masks_path = glob(os.path.join(path, "masks/*"))
#     images, masks, locations = [], [], []
#     for image_path, mask_path in zip(images_path, masks_path):
#         for i in range(num):
#             location = np.random.randint(patch_size/2 - 1, 999 - patch_size/2, (2,))
#             images.append(image_path)
#             masks.append(mask_path)
#             locations.append(location)
#     return images, masks, locations

In [ ]:
# images, masks, locations = load_data(path, patch_size=256, num=250)

In [ ]:
# locations[0][1]

In [6]:
def get_raster(raster_path):
    ds = gdal.Open(raster_path)
    data = np.empty((ds.RasterYSize, ds.RasterXSize, ds.RasterCount))
    for b in range(1, ds.RasterCount + 1):
        band = ds.GetRasterBand(b).ReadAsArray()
        data[:, :, b-1] = band
    if data.shape[-1] > 1:
        data = norma_data(data, norma_methods='min-max')   
    return data

In [7]:
def norma_data(data, norma_methods="z-score"):
    arr = np.empty(data.shape)
    for i in range(data.shape[-1]):
        array = data.transpose(2, 0, 1)[i, :, :]
        mins, maxs, mean, std= np.percentile(array, 1), np.percentile(array, 99), np.mean(array), np.std(array)
        if norma_methods == "z-score":
            new_array = (array-mean)/std
        else:
            new_array = np.clip(2*(array-mins)/(maxs-mins), 0, 1)
        arr[:, :, i] = new_array
    return arr

In [47]:
images_path, masks_path = load_data(path)

In [48]:
datasets = tf.data.Dataset.from_tensor_slices((images_path, masks_path))

In [49]:
print(datasets)

<TensorSliceDataset shapes: ((), ()), types: (tf.string, tf.string)>


In [50]:
def pare_fun(x, y):
    def f(x, y):
        x = x.decode()
        y = y.decode()
    
        x = get_raster(x)
        y = get_raster(y)
        return x, y
    
    image, mask = tf.numpy_function(f, [x, y], [tf.double, tf.double])
    image.set_shape([1000, 1000, 7])
    mask.set_shape([1000, 1000, 1])
    return image, mask

# def pare_fun(x, y, z):
#     patch_size = 256
#     def f(x, y, z):
#         x = x.decode()
#         y = y.decode()
    
#         x = get_raster(x)
#         y = get_raster(y)
        
#         h, w = z[0], z[1]
#         d1 = int(patch_size/2 - 1)
#         d2 = int(patch_size - d1)
        
#         x = x[h-d1: h+d2, w-d1: w+d2, :]
#         y = y[h-d1: h+d2, w-d1: w+d2]  
#         return x, y, z
    
#     image, mask, location = tf.numpy_function(f, [x, y, z], [tf.double, tf.double, tf.int32])
#     image.set_shape([patch_size, patch_size, 7])
#     mask.set_shape([patch_size, patch_size, 1])
#     return image, mask, location

In [51]:
dataset = list(datasets.map(pare_fun))

In [52]:
len(dataset)

35

In [57]:
def dataset_generator(ds, width, num):
    images_patch = np.zeros((len(ds)*num, width, width, 7))
    masks_patch = np.zeros((len(ds)*num, width, width, 1))
    j = 0
    for images, masks in dataset:
        for i in range(num):
            location = np.random.randint(width/2 - 1, 999 - width/2, (2,))
            h, w = location[0], location[1]
            d1 = int(width/2 - 1)
            d2 = int(width - d1)
            images_patch[j*num + i] = images[h-d1: h+d2, w-d1: w+d2, :]
            masks_patch[j*num + i] = masks[h-d1: h+d2, w-d1: w+d2]
        j = j + 1  
    yield images_patch, masks_patch


In [58]:
dataset_generator = dataset_generator(dataset, 256, 20)

In [59]:
for images_patch, masks_patch in dataset_generator:
    print(images_patch.shape, masks_patch.shape)

(700, 256, 256, 7) (700, 256, 256, 1)


In [ ]:
def get_patches(image_path, height, width, num):
    image = get_raster(image_path)
    if image.shape[-1] > 1:
        image = norma_data(image, norma_methods='min-max')


def random_sample(m):
    x = np.random.randint(m/2 - 1, 999 - m/2)
    y = np.random.randint(m/2 - 1, 999 - m/2)
    location = (x, y)
    return location

In [ ]:
class Treedatasets(object):
    def __init__(self, image_path, mask_path, patch_size, number):
        self.image_path = image_path
        self.mask_path = mask_path
        self.patch_size = patch_size
        self.number = number
        self.images = list(map(get_raster, image_path))
        self.masks = list(map(get_raster, mask_path))
        
    
    def __len__(self):
        return len(self.image_path) * self.number

    def __getitem__(self, item):
        image_id = item // self.number
        location = random_sample(self.patch_size)
        image, mask = self.images[image_id], self.masks[image_id]
        
        h, w = location[0], location[1]
        d1 = int(self.patch_size/2 - 1)
        d2 = int(self.patch_size - d1)
        
        patch_image = image[h-d1: h+d2, w-d1: w+d2, :]
        patch_mask = mask[h-d1: h+d2, w-d1: w+d2][:, :, 0]
        
        sample = {'image': patch_image, 'mask': patch_mask,
                  'patch': location}
        return sample

In [ ]:
Treedatasets = Treedatasets(image_path=images, mask_path=masks, patch_size=256, number=1)

In [ ]:
first_sample = Treedatasets[6]

In [ ]:
plt.imshow(first_sample['mask'][:, :])
plt.show()

In [ ]:
plt.imshow(first_sample['image'][:, :, 0:3])
plt.show()

# Model Construction

In [ ]:
def res_block(x, nb_filters, strides):
    res_path = BatchNormalization()(x)
    res_path = Activation(activation='relu')(res_path)
    res_path = Conv2D(filters=nb_filters[0], kernel_size=(3, 3), padding='same', strides=strides[0])(res_path)
    res_path = BatchNormalization()(res_path)
    res_path = Activation(activation='relu')(res_path)
    res_path = Conv2D(filters=nb_filters[1], kernel_size=(3, 3), padding='same', strides=strides[1])(res_path)

    shortcut = Conv2D(nb_filters[1], kernel_size=(1, 1), strides=strides[0])(x)
    shortcut = BatchNormalization()(shortcut)

    res_path = add([shortcut, res_path])
    return res_path

In [ ]:
def encoder(x):
    to_decoder = []

    main_path = Conv2D(filters=64, kernel_size=(3, 3), padding='same', strides=(1, 1))(x)
    main_path = BatchNormalization()(main_path)
    main_path = Activation(activation='relu')(main_path)

    main_path = Conv2D(filters=64, kernel_size=(3, 3), padding='same', strides=(1, 1))(main_path)

    shortcut = Conv2D(filters=64, kernel_size=(1, 1), strides=(1, 1))(x)
    shortcut = BatchNormalization()(shortcut)

    main_path = add([shortcut, main_path])
    # first branching to decoder
    to_decoder.append(main_path)

    main_path = res_block(main_path, [128, 128], [(2, 2), (1, 1)])
    to_decoder.append(main_path)

    main_path = res_block(main_path, [256, 256], [(2, 2), (1, 1)])
    to_decoder.append(main_path)

    return to_decoder

In [ ]:
def decoder(x, from_encoder):
    main_path = UpSampling2D(size=(2, 2))(x)
    main_path = concatenate([main_path, from_encoder[2]], axis=3)
    main_path = res_block(main_path, [256, 256], [(1, 1), (1, 1)])

    main_path = UpSampling2D(size=(2, 2))(main_path)
    main_path = concatenate([main_path, from_encoder[1]], axis=3)
    main_path = res_block(main_path, [128, 128], [(1, 1), (1, 1)])

    main_path = UpSampling2D(size=(2, 2))(main_path)
    main_path = concatenate([main_path, from_encoder[0]], axis=3)
    main_path = res_block(main_path, [64, 64], [(1, 1), (1, 1)])

    return main_path

In [ ]:
def build_res_unet(input_shape):
    inputs = Input(shape=input_shape)

    to_decoder = encoder(inputs)

    path = res_block(to_decoder[2], [512, 512], [(2, 2), (1, 1)])

    path = decoder(path, from_encoder=to_decoder)

    path = Conv2D(filters=1, kernel_size=(1, 1), activation='sigmoid')(path)

    return Model(inputs=inputs, outputs=path)

In [ ]:
input_shape = (512, 512, 8)

In [ ]:
model = build_res_unet(input_shape=input_shape)
model.summary()

# Model Compile

In [ ]:
# Optimizer, Loss and metrics

# Model Training